Time Calibration of the Mu2e Calorimeter

by Giacinto boccia

version 0.1 | 2024-09-05

In [1]:
import numpy as np
import awkward as ak
import uproot
import quantities as pq
from concurrent.futures import ProcessPoolExecutor

In [2]:
#Cut parameters
HITNUM_CUT = 1
Q_MIN_CUT = 4000
Q_MAX_CUT = 8000
COS_THETA_CUT = 0.9
CHI_ON_NDF_CUT = 2000

In [3]:
#Input
hits_path = input("Hits file to process:")
cal_path = input("Starting caibration file:") or False
n_runs = input("Iterations to perform:")
#Name of the tree inside the file
hits_path += ":sidet"

In [4]:
#Opening the calibration start-point file
if cal_path:
    print("Starting calibration file not yet supported")
    cal_corection = pq.Quantity(np.zeros((36, 28, 2)), 'ns')
else:
    #The time-correction array is [rows, columns, SiPM] shaped
    cal_corection = pq.Quantity(np.zeros((36, 28, 2)), 'ns')

In [16]:
#Loading the tree, start by defining custom event class that can compute residurals
class Cosmic(ak.Record):     
    def t_residuals(self) -> dict | None:
        #returns a dictionary with '(row, col, sipm)' as key and the residual as value
        #Filters on the hit charge 
        q_min_fitler = self.Qval > Q_MIN_CUT
        q_max_fitler = self.Qval < Q_MAX_CUT
        q_filter = q_max_fitler & q_min_fitler
        if self.slope:
            #Select events that have a not "None" slope. For each hit return the time residual
            cos_theta = 1 / np.sqrt(1 + self.slope ** 2)
            t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
            t_res = dict()
            for i, [row, col, sipm] in enumerate(zip(self.iRow[q_filter],
                                                     self.iCol[q_filter],
                                                     self.SiPM[q_filter])):
                #Apply current time corrections
                t_arr[i] -= cal_corection[row, col, sipm]
            #I can't really understand why this np.average operation is losing the ns, 
            #assigning them again schouldn't be necessary, but it is
            t_0_ev = pq.Quantity(np.average(t_arr, weights= self.Qval[q_filter]), 'ns')
            for time, y, row, col, sipm in zip(t_arr, 
                                               self.Yval[q_filter],
                                               self.iRow[q_filter],
                                               self.iCol[q_filter],
                                               self.SiPM[q_filter]):
                #Each residual is stored in the dictionary with '(row, col, sipm)' as key
                y = y * pq.cm
                residual = time + y / (pq.c * cos_theta) - t_0_ev
                t_res[str((int(row), int(col), int(sipm)))] = residual.item()
            return t_res
        else:
            return None                
ak.behavior["cosmic"] = Cosmic
        
#Loading tree in an array structure, we only need some of the branches
branches = ("nrun", "nsubrun", "evnum", "nHits", "iRow", "iCol", "SiPM", "Xval", "Yval", "Qval", "Tval", "templTime")
with uproot.open(hits_path) as file:
    tree = file.arrays(filter_name = branches, entry_stop= 1000)
    
#Now change the array so that it uses the custom class defned above
tree = ak.Array(tree, with_name= "cosmic")

In [17]:
#To get the parameters, each event is fitted
def linear_fit(event) -> dict[str, np.double | int]:
    q_min_fitler = event.Qval > Q_MIN_CUT
    q_max_fitler = event.Qval < Q_MAX_CUT
    x_arr = event.Xval[q_min_fitler & q_max_fitler]
    y_arr = event.Yval[q_min_fitler & q_max_fitler]
    if len(x_arr) > 1:
        #Events that are not empty
        if np.max(x_arr) - np.min(x_arr) > 34.4:
            #Events that are not vertical
            [slope, intercept], residuals, _, _, _ = np.polyfit(x_arr, y_arr, deg= 1, full= True)
            chi_sq : np.double = np.sum(residuals)
            ndf = event.nHits - 2
            return {'vertical' : False, 'slope' : slope, 'intercept' : intercept, 'chi_sq' : chi_sq, 'ndf' : ndf}
        else:
            #Vertical events get flagged
            return {'vertical': True, 'slope' : None, 'intercept' : None, 'chi_sq' : None, 'ndf' : None}
    else:
        return None
        
with ProcessPoolExecutor() as executor:
    fit_results = list(executor.map(linear_fit, tree))
fit_results = ak.Array(fit_results)

In [18]:
#Add the fit results to the tree
if hasattr(fit_results, 'vertical'):
    tree = ak.with_field(tree, fit_results.vertical, "vertical")
    tree = ak.with_field(tree, fit_results.slope, "slope")
    tree = ak.with_field(tree, fit_results.intercept, "intercept")
    tree = ak.with_field(tree, fit_results.chi_sq, "chi_sq")
    tree = ak.with_field(tree, fit_results.ndf, "ndf")

In [19]:
#Apply the filters and compute the residuals
slope_cut = np.sqrt(1 / COS_THETA_CUT - 1)
filters = {'n_min' : tree.nHits > HITNUM_CUT,
           'slope_min' : abs(tree.slope) > slope_cut,
           'chi_sq' : tree.chi_sq / tree.ndf < CHI_ON_NDF_CUT}
filtered_events = tree[filters['n_min'] & filters['slope_min'] & filters['chi_sq']]
filtered_events = ak.drop_none(filtered_events)

#Get the residuals
def get_t_residuals(event) -> dict | None:
    #Just a wrapper since the executor wants a funciton and not a method
    return event.t_residuals()
with ProcessPoolExecutor() as executor:
    residuals = list(executor.map(get_t_residuals, filtered_events))
residuals = ak.Array(residuals)

residuals

/tmp/ipykernel_18796/3202369539.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
/tmp/ipykernel_18796/3202369539.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
/tmp/ipykernel_18796/3202369539.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_arr = np.array(self.Tval[q_filter] + self.templTime[q_filter]) * pq.ns
/tmp/ipykernel_18796/3202369539.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __arra

<Array [{'(6, 25, 1)': -14.7, ...}, ..., {...}] type='76 * {"(6, 25, 1)": ?...'>